# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,10.06,57,40,11.32,CL,1699133223
1,1,edinburgh of the seven seas,-37.0676,-12.3116,12.25,65,99,11.41,SH,1699133223
2,2,dankov,53.2515,39.1555,7.90,82,100,7.88,RU,1699133223
3,3,grytviken,-54.2811,-36.5092,-0.61,73,6,2.68,GS,1699133223
4,4,hafar al-batin,28.4328,45.9708,23.32,56,0,3.09,SA,1699133223


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#%%capture --no-display
# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_df=city_data_df.loc[(city_data_df['Cloudiness']==0) & (city_data_df['Max Temp']<=25)]
vacation_df_country=vacation_df[vacation_df['Country'].isin(['US','CN','IN','FR','DZ','RU','AR','MX'])]
vacation_df_country_temp=vacation_df_country[vacation_df_country['Max Temp']>10]
vacation_df_country_temp
# Drop any rows with null values
vacation_df_country_temp.dropna()

# Display sample data
vacation_df_country_temp

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,31,hamilton,39.1834,-84.5333,19.86,46,0,5.14,US,1699132764
83,83,bethel,41.3712,-73.4140,14.05,56,0,0.00,US,1699133238
152,152,venado tuerto,-33.7456,-61.9688,22.47,45,0,4.35,AR,1699133251
209,209,timimoun,29.2639,0.2310,22.21,15,0,2.57,DZ,1699133261
213,213,hooper,41.1638,-112.1224,19.83,39,0,3.60,US,1699133246
230,230,greenville,35.6127,-77.3663,22.02,55,0,2.57,US,1699133203
244,244,chester,40.1254,-75.8749,15.47,57,0,0.45,US,1699133122
269,269,childress,34.4264,-100.2040,24.14,41,0,2.57,US,1699133270
311,311,glendive,47.1053,-104.7125,12.79,50,0,3.09,US,1699133276
323,323,red hill,33.9996,-79.2581,24.39,58,0,3.40,US,1699133278


### Step 3: Create a new DataFrame called `hotel_df`.

In [48]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = vacation_df_country_temp.iloc[: , [1, 2, 3, 5, 8]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5,"Hotel Name",'')

# Display sample data
hotel_df.set_index("City")

,Lat,Lng,Humidity,Country,Hotel Name
City,,,,,
hamilton,39.1834,-84.5333,46,US,
bethel,41.3712,-73.4140,56,US,
venado tuerto,-33.7456,-61.9688,45,AR,
timimoun,29.2639,0.2310,15,DZ,
hooper,41.1638,-112.1224,39,US,
greenville,35.6127,-77.3663,55,US,
chester,40.1254,-75.8749,57,US,
childress,34.4264,-100.2040,41,US,
glendive,47.1053,-104.7125,50,US,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [61]:
# Set parameters to search for a hotel

limit = 20
radius = 10000
#filters = f"circle:{Lng},{Lat},{radius}"
#categories= accommodation.hotel

params = {
    
    
    "limit":limit,
    "radius":radius,
    "apiKey":geoapify_key,
   
}    
    

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():  
    
    latitude=hotel_df["Lat"]
    longitude=hotel_df["Lng"]
    city=hotel_df["City"]
    params["location"] = f'{latitude},{longitude}'
    params["categories"] = f'{accommodation.hotel}'
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #params["filter"] = # YOUR CODE HERE
    #params["bias"] = # YOUR CODE HERE
    
    
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    #name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


NameError: name 'accommodation' is not defined

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)